### Source of this code
##https://medium.freecodecamp.org/a-beginners-guide-to-training-and-deploying-machine-learning-models-using-python-48a313502e5a

In [1]:
import pandas as pd
import requests,json

In [2]:
## Setting the working directory
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\red-wine-quality-cortez-et-al-2009')

In [6]:
## Loading the dataset
df=pd.read_csv('winequality-red.csv')

In [7]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
## getting only the column called quality
label=df.quality

In [12]:
label.unique()

array([5, 6, 7, 4, 8, 3], dtype=int64)

In [9]:
## getting every column except quality
features=df.drop('quality',axis=1)

In [10]:
import pandas as pd
import numpy as np
from sklearn import linear_model

In [13]:
#defining our linear regression estimator and training it with our wine data
regr = linear_model.LinearRegression()
regr.fit(features, label)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [14]:
print(regr.predict([[7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4]]).tolist())

[5.065570349074925]


In [15]:
features.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [18]:
import pickle
#creating and training a model
regr = linear_model.LinearRegression()
regr.fit(features, label)

#serializing our model to a file called model.pkl

pickle_out = open("C:\\Users\\prudi\\Documents\\GitHub\\Code\\Projects\\WineQuality\\model.pickle_sep_2019","wb")
pickle.dump(regr, pickle_out)
pickle_out.close()


In [19]:
url='http://localhost:9000/api'
data=json.dumps({'fixed acidity':7.4,'volatile acidity':0.70,'citric acid':0.00,'residual sugar':1.9,'chlorides':0.076,'free sulfur dioxide':11.0,'total sulfur dioxide':34.0,'density':0.9978,'pH':3.51,'sulphates':0.56,'alcohol':9.4})
r=requests.post(url,data)
print(r)
#print(r.json())

<Response [200]>


In [150]:
import os 
import json
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import roc_auc_score, mean_squared_error

In [151]:
import os
# Office
#os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\Loan Prediction')
#data = pd.read_csv('LoanTrain.csv')

# Personal
os.chdir('C:\\Users\\Admin\\Desktop\\Data\\Loan Prediction')
data = pd.read_csv('LoanTrain.csv')

In [152]:
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

class PreProcessing(BaseEstimator, TransformerMixin):


    def __init__(self):
        pass

    def transform(self, df):

        pred_var = ['Gender','Married','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']
        
        df = df[pred_var]
        
        #f['Dependents'] = df['Dependents'].fillna(0)
        df['Self_Employed'] = df['Self_Employed'].fillna('No')
        df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(self.term_mean_)
        df['Credit_History'] = df['Credit_History'].fillna(1)
        df['Married'] = df['Married'].fillna('No')
        df['Gender'] = df['Gender'].fillna('Male')
        df['LoanAmount'] = df['LoanAmount'].fillna(self.amt_mean_)
        
        #loading a MINMAX Scaler from a file called minmax_pickle.pkl
        MinMaxpickle_in = open("C:/Users/Admin/Desktop/Data/Loan Prediction/minmax_pickle.pkl","rb")
        MinmaxScaler_dict = pickle.load(MinMaxpickle_in)
        df['ApplicantIncome']=pd.DataFrame(MinmaxScaler_dict['ApplicantIncome'].fit_transform(df[['ApplicantIncome']]))
        
        #loading a One Hot Encoder Scaler from a file called binarizer_pickle.pkl
        Onehotpickle_in = open("C:/Users/Admin/Desktop/Data/Loan Prediction/binarizer_pickle.pkl","rb")
        Onehot_dict = pickle.load(Onehotpickle_in)
        d1=pd.DataFrame(Onehot_dict['Self_Employed'].fit_transform(df[['Self_Employed']]))
        d2=pd.DataFrame(Onehot_dict['Married'].fit_transform(df[['Married']]))
        d3=pd.DataFrame(Onehot_dict['Gender'].fit_transform(df[['Gender']]))
        d=pd.concat([d1,d2,d3],axis=1)
        col=['Self_Employed_0','Married_0','Gender_0']
        d.columns=col
        df.drop(['Self_Employed','Married','Gender'],axis=1,inplace=True)
        df=pd.concat([df,d],axis=1)
        
        ## Loading a Label encoder scaler from a file called labelencoder_pickle
        Labelencoderpickle_in = open("C:/Users/Admin/Desktop/Data/Loan Prediction/labelencoder_pickle.pkl","rb")
        Labelencoder_dict = pickle.load(Labelencoderpickle_in)
        df['Education']=pd.DataFrame(Labelencoder_dict['Education'].fit_transform(df[['Education']]))
        df['Property_Area']=pd.DataFrame(Labelencoder_dict['Property_Area'].fit_transform(df[['Property_Area']]))
        df['Credit_History']=pd.DataFrame(Labelencoder_dict['Credit_History'].fit_transform(df[['Credit_History']]))
        #df['Dependents']=pd.DataFrame(Labelencoder_dict['Dependents'].fit_transform(df[['Dependents']]))
        
        
       
        return df.as_matrix()

    def fit(self, df, y=None, **fit_params):
        
        self.term_mean_ = df['Loan_Amount_Term'].mean()
        self.amt_mean_ = df['LoanAmount'].mean()
        return self

In [153]:
preprocess = PreProcessing()

In [154]:
preprocess.fit(data)

PreProcessing()

In [155]:
data_transformed = preprocess.transform(data)

In [156]:
pred_var = ['Education','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area','Self_Employed_0','Married_0','Gender_0']
data_transformed=pd.DataFrame(data_transformed,columns=pred_var)
data_transformed.head()
#data_transformed['Dependents'].unique()

,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Self_Employed_0,Married_0,Gender_0
0,0.0,0.070489,0.0,146.412162,360.0,1.0,2.0,0.0,0.0,1.0
1,0.0,0.054830,1508.0,128.000000,360.0,1.0,0.0,0.0,1.0,1.0
2,0.0,0.035250,0.0,66.000000,360.0,1.0,2.0,1.0,1.0,1.0
3,1.0,0.030093,2358.0,120.000000,360.0,1.0,2.0,0.0,1.0,1.0
4,0.0,0.072356,0.0,141.000000,360.0,1.0,2.0,0.0,0.0,1.0


In [ ]:
#loading a model from a file called model.pkl
pickle_in = open("C:\\Users\\Admin\\Documents\\GitHub\\Code\\Projects\\WineQuality\\model.pickle","rb")
example_dict = pickle.load(pickle_in)